In [17]:
import csv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

In [18]:
# Date Definitions
beforeflood_date = "2019-03-11"
duringflood_date = "2019-03-23"

# Polarization Selection
def setup():
    return {
        "input": {
            "bands": ["VV"]
        },
        "output": {"bands": 3},
        "mosaicking": "ORBIT"
    }

# Preprocessing of Scenes
def pre_process_scenes(collections, allowed_dates):
    # Filter scenes based on allowed dates
    collections["scenes"]["orbits"] = [
        orbit for orbit in collections["scenes"]["orbits"]
        if orbit["dateFrom"].split("T")[0] in allowed_dates
    ]
    return collections

# Flood Mapping Calculation
def calc_fm(sample):
    return 1.5 * sample["VV"]

# Date Formatting Helper
def date_format(d):
    return d.strftime("%Y-%m-%d")

# Evaluate Pixel
def evaluate_pixel(samples, scenes):
    out_be = calc_fm(samples[1])  # before-flood image
    out_du = calc_fm(samples[0])  # during-flood image
    return [out_be, out_du, out_du]

# Example Collections and Samples
collections = {
    "scenes": {
        "orbits": [
            {"dateFrom": "2019-03-11T00:00:00"},
            {"dateFrom": "2019-03-23T00:00:00"}
        ]
    }
}

samples = [
    {"VV": 0.5},  # During flood sample
    {"VV": 0.3}   # Before flood sample
]

# Main Execution
allowed_dates = [beforeflood_date, duringflood_date]
collections = pre_process_scenes(collections, allowed_dates)
result = evaluate_pixel(samples, collections)
print("Result:", result)

Result: [0.44999999999999996, 0.75, 0.75]


The result represents the processed radar backscatter intensity (VV band) from satellite imagery, calculated before and during a flood event. Here's what each value implies:

The result represents the processed radar backscatter intensity (VV band) from satellite imagery, calculated before and during a flood event. Here's what each value implies:

1. out_be (Before Flood):
Value: 0.45
This is the radar signal's intensity (scaled by a factor of 1.5) from the satellite image taken before the flood.
It reflects the surface conditions, like soil moisture, vegetation, or urban structures, before the flood occurred.

2. out_du (During Flood):
Value: 0.75 (repeated twice for output format consistency).
This is the radar signal's intensity (scaled by a factor of 1.5) from the satellite image taken during the flood.
The higher value compared to out_be could indicate changes in the surface, such as the presence of water, which has different radar reflectivity compared to dry land.


Comparison (out_be vs out_du):
The increase from 0.45 to 0.75 suggests a significant change in the surface reflectivity, likely due to water inundation during the flood. Water surfaces typically have a strong backscatter signal in radar data, which is why the value is higher during the flood.This data helps in flood detection and mapping:

Identify flooded areas by comparing radar intensities before and during the event.
Generate flood extent maps for disaster response and damage assessment.
If you add more processing (e.g., masking regions where the difference exceeds a threshold), you could create binary maps that show flooded vs non-flooded areas.

Extended script to calculate additional satellite-based features and prepare a dataset suitable for ML models.

In [19]:
# Additional helper for change detection
def calculate_change(before, during):
    return during - before

# Add more features (e.g., NDWI, NDVI, elevation)
def calculate_ndwi(nir, green):
    return (green - nir) / (green + nir)

def calculate_ndvi(nir, red):
    return (nir - red) / (nir + red)

# Extend evaluate_pixel to include additional features
def evaluate_pixel(samples, scenes):
    # Before and during flood
    before = calc_fm(samples[1])  # Before flood VV
    during = calc_fm(samples[0])  # During flood VV

    # Change detection
    change_vv = calculate_change(before, during)

    # Add NDVI and NDWI as dummy data (replace with actual data extraction logic)
    ndvi = calculate_ndvi(samples[1].get("NIR", 0.4), samples[1].get("RED", 0.2))  # Before flood
    ndwi = calculate_ndwi(samples[1].get("NIR", 0.4), samples[1].get("GREEN", 0.3))  # Before flood

    # Terrain features (dummy elevation, replace with actual DEM data)
    elevation = scenes.get("elevation", 500)  # Example elevation in meters

    return {
        "before_vv": before,
        "during_vv": during,
        "change_vv": change_vv,
        "ndvi": ndvi,
        "ndwi": ndwi,
        "elevation": elevation
    }

# Example data for ML
samples = [
    {"VV": 0.5, "NIR": 0.6, "RED": 0.2, "GREEN": 0.3},  # During flood
    {"VV": 0.3, "NIR": 0.4, "RED": 0.2, "GREEN": 0.3}   # Before flood
]

scenes = {
    "elevation": 450  # Example elevation
}

# Generate data for ML
data_point = evaluate_pixel(samples, scenes)
print("ML Features:", data_point)

ML Features: {'before_vv': 0.44999999999999996, 'during_vv': 0.75, 'change_vv': 0.30000000000000004, 'ndvi': 0.3333333333333333, 'ndwi': -0.1428571428571429, 'elevation': 450}


In [20]:
dataset = []
areas = ["Area1", "Area2"]  # Replace with real area IDs
for area in areas:
    for time in ["before_flood", "during_flood"]:
        # Simulated or fetched satellite data
        #samples = get_satellite_samples(area, time)  # Replace with actual data fetching
        #scenes = get_scene_metadata(area)           # Replace with real metadata
        features = evaluate_pixel(samples, scenes)
        
        # Add a flood risk label (manually or based on logic)
        if time == "during_flood":  # Assuming "during_flood" corresponds to flood events
            features["flood_risk"] = 1
        else:
            features["flood_risk"] = 0
        
        # Add geographic and temporal metadata
        features["area"] = area
        features["time"] = time
        dataset.append(features)

# Save to CSV
with open("flood_risk_data.csv", "w", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=dataset[0].keys())
    writer.writeheader()
    writer.writerows(dataset)

In [21]:
# Load the dataset
data = pd.read_csv("flood_risk_data.csv")

# Prepare features and labels
X = data[["before_vv", "during_vv", "change_vv", "ndvi", "ndwi", "elevation"]]
y = data["flood_risk"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
print("Model trained!")

Model trained!


Once the model is trained, it can be used to predict flood risks in real time based on new satellite data and environmental conditions. This can help in early warning systems, disaster response planning, and risk mitigation strategies.